In [1]:
from keras.models import * 
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.utils import multi_gpu_model
import tensorflow as tf

Using TensorFlow backend.


In [2]:
input_shape = (299, 299, 3)
nb_classes = 20
nb_gpus = 2

In [3]:
with tf.device('/cpu:0'):
    input_tensor = Input(input_shape)
    x = Lambda(xception.preprocess_input)(input_tensor)

    base_model = Xception(input_tensor=x, weights=None, include_top=False)
    m_out = base_model.output
    p_out = GlobalAveragePooling2D()(m_out)
    p_out = Dropout(0.5)(p_out)
    prediction = Dense(nb_classes, activation='softmax', name="prediction")(p_out)
    model = Model(inputs=base_model.input, outputs=prediction)

if nb_gpus > 1:
    model = multi_gpu_model(model, gpus=nb_gpus)
    model.compile(optimizer='Adadelta', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
model_1 (Model)                 (None, 20)           20902460    lambda_2[0][0]                   
                                                                 lambda_3[0][0]                   
__________

In [4]:
model.load_weights("weights_036_0.1710.hdf5")

In [ ]:
batch_size = 64
img_size = (299, 299)
test_path = "/home/nvme/CELLS_hls_299/test"

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory(test_path, 
                                         img_size, 
                                         shuffle=False,
                                         batch_size=batch_size)
nb_tests = test_generator.samples
y_test = test_generator.classes
y_pred = model.predict_generator(test_generator, steps=len(test_generator), verbose=1)

Found 13762 images belonging to 20 classes.
204/216 [===========================>..] - ETA: 2s

In [ ]:
print(nb_tests, len(y_test), len(y_pred))
print(y_test.shape, y_test[0])
print(y_pred.shape, y_pred[0])

In [ ]:
# confusion matrix
import numpy as np
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_test, y_pred.argmax(axis=1))
print(conf_matrix)

In [ ]:
class_map = test_generator.class_indices
print(class_map)
classes = sorted(class_map.items(), key=lambda kv: kv[1])
print(classes)
classes = [entry[0] for entry in classes]
print(classes)

In [ ]:
# AUC ROC

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from keras.utils import to_categorical
from sklearn.metrics import roc_curve, auc

# one-hot encode y_test
y_test_ = to_categorical(y_test)
print(y_test_.shape, y_test_[0])

fig, ax = plt.subplots(1,1, figsize = (9, 9))
for i,class_i in enumerate(classes):
    fpr, tpr, thresholds = roc_curve(y_test_[:,i], y_pred[:,i])
    ax.plot(fpr, tpr, label="{0} (AUC:{1:0.4f})".format(class_i, auc(fpr, tpr)))
ax.legend()
ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")